In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import torch, sys, os
print("python:", sys.executable)
print("torch:", torch.__version__)
print("cuda_available:", torch.cuda.is_available())
print("cuda_device_count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("current device:", torch.cuda.current_device(), "name:", torch.cuda.get_device_name(0))

python: /home/hychuiaa/anaconda3/envs/ipmgnn/bin/python
torch: 2.0.0
cuda_available: True
cuda_device_count: 4
current device: 0 name: NVIDIA GeForce RTX 2080 Ti


In [15]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Example usage:
# x = torch.randn(3,3)
# x = x.to(device)
# For a model:
# model = MyModel(...)
# model.to(device)
# When creating tensors with torch.from_numpy(...), then call .to(device) as in the notebook:
# torch.from_numpy(A).to(torch.float).to(device)

In [16]:
from solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning
import warnings
import numpy as np
from functools import partial

from pathlib import Path

from generate_instances import generate_random_lp, generate_setcover, Graph, generate_indset, generate_cauctions, generate_capacited_facility_location

In [17]:
rng = np.random.RandomState(1)

In [ ]:
bounds = (0., 1.)

In [19]:
root = os.getcwd() + '/d/fac6'
root

'/import/home/hychuiaa/FYP_file/IPM_MPNN/d/fac6'

In [20]:
!mkdir -p /home/tony/FYP_file/IPM_MPNN/d/fac6
# !mkdir /mnt/d/fac6/raw
# !mkdir /mnt/d/fac6/processed_1restarts_0lap_8steps_upper_1.0
print("os.getcwd():", os.getcwd())

mkdir: cannot create directory ‘/home/tony’: Permission denied
os.getcwd(): /import/home/hychuiaa/FYP_file/IPM_MPNN


### Setcover

In [21]:
density=0.01
nrows_l = 700
nrows_u = 701
ncols_l = 700
ncols_u = 701

surrogate_gen = partial(generate_setcover, nrows_l=nrows_l, nrows_u=nrows_u, ncols_l=ncols_l, ncols_u=ncols_u, density=density, rng=rng)

### Indset

In [22]:
def surrogate_gen():
    # nnodes = rng.randint(10, 20)
    # nnodes = rng.randint(500, 700)
    nnodes = 550
    graph = Graph.barabasi_albert(number_of_nodes=nnodes, affinity=2, random=rng)
    A, b, c = generate_indset(graph=graph, nnodes=nnodes)
    return A, b, c

### Cauctions

In [23]:
def surrogate_gen():
    # n_items=rng.randint(50, 80)
    # n_bids=rng.randint(50, 80)
    n_items = 500
    n_bids = 600
    A, b, c = generate_cauctions(n_items=n_items, n_bids=n_bids, rng=rng, min_value=0.01, max_value=0.1)
    return A, b, c

### Facilities

In [24]:
def surrogate_gen():
    n_customers = 25
    n_facilities = 35
    ratio = 5
    A, b, c = generate_capacited_facility_location(n_customers=n_customers, n_facilities=n_facilities, ratio=ratio, rng=rng)
    return A, b, c

## Random LP

In [25]:
def surrogate_gen():
    nvars = 5000
    nconstraints = 5000
    density = 1
    A, b, c = generate_random_lp(nvars=nvars, nconstraints=nconstraints, density=density, rng=rng)
    return A, b, c

## create

In [ ]:
warnings.filterwarnings("error")
#cauction
ips = []
pkg_idx = 0
success_cnt = 0
fail_cnt = 0

max_iter = 15000
num = 100

for i in tqdm(range(max_iter)):
    A, b, c = surrogate_gen()
    
    try:
        A_eq = None
        b_eq = None
        A_ub = A
        b_ub = b
        res = linprog(c, 
                A_ub=A_ub,
                b_ub=b_ub,
                A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='interior-point')
        print("Size of A:", A.shape)
    except (LinAlgWarning, OptimizeWarning, AssertionError):
        fail_cnt += 1
        continue
    else:
        if res.success and not np.isnan(res.fun):
            ips.append((torch.from_numpy(A).to(torch.float).to(device), torch.from_numpy(b).to(torch.float).to(device), torch.from_numpy(c).to(torch.float).to(device)))
            success_cnt += 1

    if len(ips) >= 1000 or success_cnt == num:
        out_path = Path(f'{root}/raw/instance_{pkg_idx}.pkl.gz')
        out_path.parent.mkdir(parents=True, exist_ok=True)
        with gzip.open(out_path, "wb") as file:
            pickle.dump(ips, file)
            pkg_idx += 1
        ips = []

    if success_cnt >= num:
        break

warnings.resetwarnings()

  0%|          | 1/15000 [01:24<353:11:48, 84.77s/it]

Size of A: (5000, 5000)


  0%|          | 2/15000 [03:00<379:46:48, 91.16s/it]

Size of A: (5000, 5000)


  0%|          | 3/15000 [04:35<388:11:19, 93.18s/it]

...

Size of A: (5000, 5000)


  0%|          | 75/15000 [1:59:20<413:21:37, 99.70s/it]

Size of A: (5000, 5000)


  1%|          | 76/15000 [2:01:06<421:48:29, 101.75s/it]

Size of A: (5000, 5000)
